# Zero to Hero : An Overwatch Hero Recommender

# Table of Contents:

1. **Data and Functions Import**  
2. **Cleaning and Reformating Data into Relevant Structure**  
3. **Role Category Implementation**  
4. **Implementing the Recommendation System**  
    4.1 Training the model  
5. **Making a Recommendation**  
    5.1 Interactive user_rating input  
    5.2 Making predictions with new user ratings

## Executive Summary
Zero to Hero is a recommendation system that offers users who play the popular eSports title: Overwatch a way to choose a hero to play with. In Overwatch, two teams of 6 players go head to head in a heated contest. With a growing pool of 31 heroes to choose from in 3 different roles, new and veteran users can benefit from this hero recommender. 

Often players will be stuck single-mindedly on a hero unwilling to choose a new hero to play. This can put enormous pressure on the team. These users will be able to get a hero recommended to them using Zero to hero. Older players who have played a long time and want to diversify their hero pool can also benefit. 

The system falls in line with the game developers encouragement to players of choosing heroes that they enjoy playing or at the very least providing some variety in their hero choices.

## 1. Data and Functions Import

In [3]:
# Import all local functions from the source py file:
from recommender_utils import *

In [4]:
# Read in the top5 heroes data for 60k users:
df_top5 = pd.read_csv('60k_Top5_heroes.csv')
df_top5

,Battle_Tag,Hero1,Hero2,Hero3,Hero4,Hero5
0,Morellio-1416,Mercy,Reinhardt,Orisa,Moira,Winston
1,Zinux-1251,Hanzo,Ana,Torbjörn,Soldier: 76,Lúcio
2,BluePillow-21338,Hanzo,Genji,Widowmaker,McCree,Tracer
3,Miyuna-21516,Widowmaker,Reinhardt,Pharah,Genji,Hanzo
4,Mood-21538,Widowmaker,Tracer,Hanzo,Ana,Zenyatta
...,...,...,...,...,...,...
64228,Dot-11883,Lúcio,Hanzo,Ana,Widowmaker,Genji
64229,hanayarawa55-1556,Ana,Zenyatta,Sigma,Moira,Lúcio
64230,blowedglass-1740,Winston,Reinhardt,Roadhog,D.Va,Hanzo
64231,FlashedDusk-2754,Genji,Ana,Reaper,Moira,Ashe


## 2. Cleaning and formatting Data

In [5]:
# Apply preprocessing and curation to clean missing values, format data as Dummies
# and apply ratings to the top5 heroes:
heroes_preprocessed = preprocessing(df_top5)
heroes_curated = apply_curation(heroes_preprocessed)

In [6]:
# Melt the dummy data into the corect format for the recommendation engine:
heroes = concat_melt(heroes_curated)
# Rename the columns of the dataframe:
heroes.rename(columns={"variable":"Hero", "value":"Ratings"}, inplace = True)
heroes

,Battle_Tag,Hero,Ratings
0,Morellio-1416,Ana,0.0
1,Zinux-1251,Ana,4.8
2,BluePillow-21338,Ana,0.0
3,Miyuna-21516,Ana,0.0
4,Mood-21538,Ana,4.4
...,...,...,...
1870783,Dot-11883,Zenyatta,0.0
1870784,hanayarawa55-1556,Zenyatta,4.8
1870785,blowedglass-1740,Zenyatta,0.0
1870786,FlashedDusk-2754,Zenyatta,0.0


## 3. Role Category Implementation

In [7]:
# Create Role category for each Hero;
tanks = ['D.Va','Orisa', 'Reinhardt', 'Roadhog', 'Sigma', 'Winston', 'Wrecking Ball', 'Zarya']
damage = ['Ashe','Bastion','Doomfist','Genji','Hanzo','Junkrat','McCree','Mei','Pharah','Reaper','Soldier: 76','Sombra','Symmetra','Torbjörn','Tracer','Widowmaker']
support = ['Ana','Baptiste','Brigitte','Lúcio','Mercy','Moira','Zenyatta']

# Create list of all heroes to use in predict function later
all_heroes = tanks+damage+support
# create roles df and apply role column adding in the role of the hero. To be used in interactive input function later
roles = heroes.copy()
roles['Role'] = ['Tank' if x in tanks else 'Damage' if x in damage else 'Support' for x in heroes['Hero']] 


## 4. Implementing the Recommendation system

In [8]:
# importing relevant libraries
from surprise import Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

# from surprise.model_selection import cross_validate
# from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
# from surprise.model_selection import GridSearchCV


#Read in Values as a Surprise Dataset:
reader = Reader(rating_scale=(1,5))

train_data = Dataset.load_from_df(heroes[['Battle_Tag', 'Hero', 'Ratings']], reader)

# Train test split the dataset 
trainset, testset = train_test_split(train_data, test_size=.25)


### 4.1 Training the Model

In [9]:
# # Instantiate KNN Baseline memory based model:
# # ⏰ This cell may take a couple of minutes to run

algobaseitems = KNNBaseline(sim_options={'name':'cosine', 'user_based':False})
algobaseitems.fit(trainset)

# Evaluating the model performance on testset and observing the model accuracy:
predictions = algobaseitems.test(testset)
accuracy.rmse(predictions)


Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.6819


1.6819185383841304

## 5. Making a Recommendation


### 5.1 Interactive User Rating Input

In [10]:
## try out the interactive user rating function here!
user_rating = hero_rater('Newuser2', roles, 5)

            Hero    Role
406143  Doomfist  Damage


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 3


            Hero     Role
126220  Baptiste  Support


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 4


           Hero    Role
1032525  Reaper  Damage


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 5


                  Hero  Role
1710326  Wrecking Ball  Tank


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 3


          Hero  Role
1782400  Zarya  Tank


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 5


In [12]:
# check the format of our user_rating
user_rating

[{'Battle_Tag': 'Newuser2', 'Hero': 'Doomfist', 'Ratings': '3'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Baptiste', 'Ratings': '4'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Reaper', 'Ratings': '5'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Wrecking Ball', 'Ratings': '3'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Zarya', 'Ratings': '5'}]

### 5.2 Making Predictions with the new user ratings


In [13]:
# Call our prediction function to predict a new hero for a new user
prediction(user_rating, heroes, algobaseitems, all_heroes, reader)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


[('Widowmaker', 1.1974269204383985),
 ('Genji', 1.1775652767757296),
 ('McCree', 1.0691791238933401),
 ('Hanzo', 1.029629563781194),
 ('Ana', 0.6265660868767509),
 ('Reinhardt', 0.5393137476385463),
 ('Roadhog', 0.5057409003590368),
 ('Tracer', 0.5000239527047926),
 ('Soldier: 76', 0.48423073234481273),
 ('D.Va', 0.449974172666582),
 ('Mercy', 0.42236149146356183),
 ('Lúcio', 0.3447034353708307),
 ('Junkrat', 0.300277597099189),
 ('Sigma', 0.1851965029540185),
 ('Pharah', 0.1758969331266661),
 ('Zenyatta', 0.15545521309556637),
 ('Moira', 0.1382095814571166),
 ('Winston', 0.09183309551636876),
 ('Orisa', 0.02054644549572951),
 ('Brigitte', -0.044967888900961234),
 ('Mei', -0.04561354522803551),
 ('Torbjörn', -0.09057296388261515),
 ('Ashe', -0.09855819214975325),
 ('Bastion', -0.10758050758338061),
 ('Symmetra', -0.14137460748663344),
 ('Sombra', -0.19395536179511463)]